### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from Configuration import *

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### View your sample population

In [5]:
sample_pop = configs.enc.initialize()
print(sample_pop)

[{'gene': array([75, 58, 35, 48, 43, 96, 86, 33,  0,  5, 52, 26, 61, 94, 23, 18, 88,
        14, 13, 85, 93, 80, 71, 20, 66,  6,  1, 53, 79,  2, 95, 16, 73, 74,
        34, 51, 29, 90, 17, 91, 21, 89, 10, 77, 55, 83, 38, 49, 28, 39, 67,
        46, 81, 42, 32, 50, 41, 15, 87, 62, 64, 82,  4, 19, 59, 25,  7, 84,
        44, 30, 24, 99, 97, 70,  8, 40, 65, 63, 27, 45, 57, 37, 12, 54, 47,
        22,  3, 78, 92, 69, 31,  9, 98, 68, 11, 76, 36, 56, 60, 72]), 'fitness': array([0])}
 {'gene': array([25, 18, 94, 30, 26, 51, 88, 20, 11,  5, 50, 59, 81, 35, 92, 60, 28,
        58, 66, 53, 40, 23, 47, 49, 89, 15, 72, 44, 62, 90, 24, 68, 32, 22,
        21, 14, 12, 42, 97, 98,  2, 34, 96, 78, 63,  4, 70, 95, 55,  8, 74,
        61, 75, 46, 84, 56, 57, 45, 17, 39, 54,  0, 16,  3, 85, 91, 99, 86,
        48, 77, 76, 27, 69, 64, 43, 19,  7, 83, 10, 33, 67,  1, 29, 37, 31,
        65, 41, 13,  9, 93, 87, 52,  6, 38, 36, 80, 71, 79, 73, 82]), 'fitness': array([0])}
 {'gene': array([67, 41, 11, 79, 71,

### Define your evaluation function

Your evaluation function should update all fitness values of an individual ```ind```. For example, the evaluation function for the single-objective Traveling Salesman Problem is defined below.

```
def eval(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() + dist[ind['gene'][-1]][0]

    return pop
```

In [6]:
def eval(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() + dist[ind['gene'][-1]][0]

    return pop

In [7]:
sample_ind = eval(sample_pop)
print(sample_ind[0])

{'gene': array([75, 58, 35, 48, 43, 96, 86, 33,  0,  5, 52, 26, 61, 94, 23, 18, 88,
       14, 13, 85, 93, 80, 71, 20, 66,  6,  1, 53, 79,  2, 95, 16, 73, 74,
       34, 51, 29, 90, 17, 91, 21, 89, 10, 77, 55, 83, 38, 49, 28, 39, 67,
       46, 81, 42, 32, 50, 41, 15, 87, 62, 64, 82,  4, 19, 59, 25,  7, 84,
       44, 30, 24, 99, 97, 70,  8, 40, 65, 63, 27, 45, 57, 37, 12, 54, 47,
       22,  3, 78, 92, 69, 31,  9, 98, 68, 11, 76, 36, 56, 60, 72]), 'fitness': array([46263])}


### Define a custom Plotly figure to display an individual's phenotypic (high-level) representation

For example, the phenotypic representation for a solution Traveling Salesman Problem is defined below as a graph network

```
def custom(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig
```

In [8]:
def custom(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig

### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [9]:
configs.eval, configs.vis = eval, custom

### Run the application

In [10]:
main_app = app_file.create_app(configs)
main_app.run_server()

Dash app running on http://127.0.0.1:8050/
